## LBPH

Local Binary Patterns Histograms

1. Radius
   - 이 값을 증가시키면, 엣지를 찾는데 어려움을 겪습니다.
   - 하지만, 더 많은 패턴을 찾을 수 있습니다.
2. Neighbors
    - 실제로 연산하게 되는 양을 의미합니다.
    - 이 값이 늘어날수록, 연산 시간이 길어집니다.
3. grid_x & grid_y
    - 셀이 많은 수록, 히스토그램의 수가 늘고 패턴을 찾기 쉬워집니다.
    - 셀이 적을 수록, 히스토그램의 수가 적고 패턴을 찾기 어렵지만 연산 속도가 빨라집니다.
4. threshold
    - 감지의 신뢰도를 의미하며, 높을 수록 안면 인식의 품질이 좋아집니다.

In [7]:
import os
import cv2
import numpy as np
from PIL import Image

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
# LBPH Rrecognizer
lbph_classifier = cv2.face.LBPHFaceRecognizer_create()
lbph_classifier.read('./lbph_classifier.yml')

In [11]:
paths = [os.path.join('yalefaces/test', f) for f in os.listdir('yalefaces/test')]
paths = [f.replace('\\', '/') for f in paths]

predictions = []
expected_outputs = []

for path in paths:
    
    image = Image.open(path).convert('L')
    image_np = np.array(image, 'uint8')
    
    prediction, _ = lbph_classifier.predict(image_np)
    expected_output = int(os.path.split(path)[1][7:9])
    
    predictions.append(prediction)
    expected_outputs.append(expected_output)

predictions = np.array(predictions)
expected_outputs = np.array(expected_outputs)

In [12]:
accuracy_score(expected_outputs, predictions)

0.6666666666666666

### 튜닝된 얼굴 인식

1. threshold = 1.7976931348623157e+308
2. radius = 1
3. neighbors = 8
4. grid_x = 8
5. grid_y = 8

In [14]:
def get_image_data():
    paths = [os.path.join('yalefaces/train', f) for f in os.listdir('yalefaces/train')]
    paths = [f.replace('\\', '/') for f in paths]
    
    faces = []
    ids = []
    
    for path in paths:
        image = Image.open(path).convert('L') # 'L'은 Single Channel을 의미합니다.
        image_np = np.array(image, 'uint8')
        
        id = int(os.path.split(path)[1][7:9])
        
        ids.append(id)
        faces.append(image_np)
    
    return np.array(ids), faces
ids, faces = get_image_data()

In [41]:
# LBPH Rrecognizer
tunning_lbph_classifier = cv2.face.LBPHFaceRecognizer_create(
    radius = 1,
    neighbors = 8,
    grid_x = 8,
    grid_y = 8
)
tunning_lbph_classifier.train(src=faces, labels=ids)
tunning_lbph_classifier.write('lbph_classifier_tunning.yml')

In [42]:
paths = [os.path.join('yalefaces/test', f) for f in os.listdir('yalefaces/test')]
paths = [f.replace('\\', '/') for f in paths]

predictions = []
expected_outputs = []

for path in paths:
    
    image = Image.open(path).convert('L')
    image_np = np.array(image, 'uint8')
    
    prediction, _ = tunning_lbph_classifier.predict(image_np)
    expected_output = int(os.path.split(path)[1][7:9])
    
    predictions.append(prediction)
    expected_outputs.append(expected_output)

predictions = np.array(predictions)
expected_outputs = np.array(expected_outputs)

In [39]:
accuracy_score(expected_outputs, predictions)

0.6333333333333333